In [18]:
import requests,zipfile, io, os, csv
import xml.etree.ElementTree as ET
import pandas as pd
import boto3

In [2]:
file_path = './case_study1_link.xml'

In [3]:
tree = ET.parse(file_path)

In [4]:
root = tree.getroot()

In [5]:
for item in root.iter('doc'):
    dict_attributes = {}
    for child in item.iter('str'):
        dict_attributes[child.attrib['name']] = child.text
    if dict_attributes['file_type'] == 'FULINS':
        response = requests.get(dict_attributes['download_link'])
        print(response.status_code, dict_attributes['download_link'], dict_attributes['file_type'])
        zipFile = zipfile.ZipFile(io.BytesIO(response.content))
        zipFile.extractall("./downloaded_xml")
    break

200 http://firds.esma.europa.eu/firds/FULINS_C_20171118_01of01.zip FULINS


In [6]:
for file in os.listdir("./downloaded_xml"):
    if 'xml' in file:
        xml_file_path = "./downloaded_xml/"+file
        tree1 = ET.parse(xml_file_path)

In [7]:
root1 = tree1.getroot()
# root1.tag

In [8]:
my_namespaces = dict([node for _, node in ET.iterparse(xml_file_path, events = ['start-ns'])])

In [9]:
print(my_namespaces)

{'': 'urn:iso:std:iso:20022:tech:xsd:DRAFT13auth.017.001.01', 'xsi': 'http://www.w3.org/2001/XMLSchema-instance'}


In [10]:
with open('Out.csv', 'w', newline='', encoding='utf-8') as csv_file:
    csvwriter = csv.writer(csv_file)
    col_names = ['FinInstrmGnlAttrbts.Id','FinInstrmGnlAttrbts.FullNm','FinInstrmGnlAttrbts.ClssfctnTp','FinInstrmGnlAttrbts.CmmdtyDerivInd',
             'FinInstrmGnlAttrbts.NtnlCcy','Issr']
    csvwriter.writerow(col_names)

    for item in root1.iterfind('.//RefData',my_namespaces):
        event = []
        event.append(item.find('./FinInstrmGnlAttrbts/Id',my_namespaces).text)
        event.append(item.find('./FinInstrmGnlAttrbts/FullNm',my_namespaces).text)
        event.append(item.find('./FinInstrmGnlAttrbts/ClssfctnTp',my_namespaces).text)
        event.append(item.find('./FinInstrmGnlAttrbts/CmmdtyDerivInd',my_namespaces).text)
        event.append(item.find('./FinInstrmGnlAttrbts/NtnlCcy',my_namespaces).text)
        event.append(item.find('./Issr',my_namespaces).text)
        csvwriter.writerow(event)
        # print(item.find('Issr',my_namespaces).text,item.find('./FinInstrmGnlAttrbts/Id',my_namespaces).text,item.find('./Issr',my_namespaces).text)

In [11]:
df = pd.read_csv('Out.csv')

In [12]:
df.shape

(57004, 6)

In [13]:
df.head()

,FinInstrmGnlAttrbts.Id,FinInstrmGnlAttrbts.FullNm,FinInstrmGnlAttrbts.ClssfctnTp,FinInstrmGnlAttrbts.CmmdtyDerivInd,FinInstrmGnlAttrbts.NtnlCcy,Issr
0,KYG936361016,VinaLand Ltd. Registered Share...,CBCGXS,False,USD,391200OUOEWDQSEJ0Y74
1,KYG936361016,VinaLand Ltd. Registered Share...,CBCGXS,False,USD,391200OUOEWDQSEJ0Y74
2,KYG936361016,"VINALAND LTD DL-,01",CBCGXS,False,USD,391200OUOEWDQSEJ0Y74
3,AU000000CQR9,Charter Hall Retail REIT Registered Units...,CBCIXS,False,AUD,391200I7OS301UELZA68
4,AU000000SGP0,Stockland Reg. Stapled Secs o.N.,CBCIXS,False,AUD,391200OUOEWDQSEJ0Y74


In [16]:
s3 = boto3.resource(
    's3',
    aws_access_key_id='YOUR_ACCESS_KEY',
    aws_secret_access_key='YOUR_SECRET_KEY'
)
# Set up S3 bucket and file name
bucket_name = 'your-bucket-name'
file_name = 'your-file-name.csv'
# Set up CSV data
with open('Out.csv','rb') as file:
    data = file
# Write CSV file to S3
s3.Object(bucket_name, file_name).put(Body=data)